## Setup

In [1]:
import pandas as pd
import unicodedata
import json
import os

In [2]:
bairros_fortaleza = [
    "aeroporto", "aerolandia", "alagadico_novo", "aldeota", "alto_da_balanca",
    "alvaro_weyne", "amadeu_furtado", "ancuri", "antonio_bezerra", "autran_nunes",
    "barra_do_ceara", "barroso", "bela_vista", "benfica", "bom_futuro",
    "bom_jardim", "bonsucesso", "cais_do_porto", "cajazeiras", "cambeba",
    "canindezinho", "carlito_pamplona", "cidade_2000", "cidade_dos_funcionarios", "coaçu",
    "coco", "conjunto_ceara_i", "conjunto_ceara_ii", "conjunto_esperanca", "conjunto_palmeiras",
    "couto_fernandes", "cristo_redentor", "curio", "damas", "democrito_rocha",
    "dende", "dias_macedo", "dionisio_torres", "dom_lustosa", "edson_queiroz",
    "ellery", "engenheiro_luciano_cavalcante", "farias_brito", "fatima", "floresta",
    "genibau", "granja_lisboa", "granja_portugal", "guajeru", "guararapes",
    "henrique_jorge", "itaoca", "itaperi", "jacarecanga", "jangurussu",
    "jardim_america", "jardim_cearense", "jardim_das_oliveiras", "jardim_guanabara", "jardim_iracema",
    "joao_xxiii", "joaquim_tavora", "joquei_clube", "jose_bonifacio", "jose_de_alencar",
    "lagoa_redonda", "manuel_dias_branco", "manoel_satiro", "maraponga", "messejana",
    "meireles", "mondubim", "monte_castelo", "montese", "moura_brasil",
    "mucuripe", "panamericano", "papicu", "parangaba", "parque_araxa",
    "parque_dois_irmaos", "parque_iracema", "parque_manibura", "parquelandia", "parreao",
    "passare", "paupina", "pedras", "pici", "pirambu",
    "planalto_ayrton_senna", "praia_de_iracema", "praia_do_futuro_i", "praia_do_futuro_ii", "prefeito_jose_walter",
    "presidente_kennedy", "presidente_vargas", "quintino_cunha", "rodolfo_teofilo", "sabiaguaba",
    "salinas", "santa_rosa", "sao_bento", "sao_gerardo", "sao_joao_do_tauape",
    "sao_jose", "sapiranga", "serrinha", "siqueira", "varjota",
    "vicente_pinzon", "vila_ellery", "vila_peri", "vila_uniao", "vila_velha"
]


In [3]:
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [4]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]
pd.set_option('display.max_colwidth', None)

In [5]:
local_folder = "../../downloads/Portalnoticiasceara" 
dados = []

In [6]:
for root, dirs, files in os.walk(local_folder):
    for file in files:
        if file.endswith('.json'):  # Verificar se é um arquivo JSON
            file_path = os.path.join(root, file)
            print(f"Lendo o arquivo: {file_path}")

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-15/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-17/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-18/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-14/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-19/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-13/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-16/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-27/metadata.json


## Recentes

In [7]:
from datetime import datetime, timedelta, timezone

def verificar_ultimas_24_horas(data_msg):
    try:
        # Converter a data da mensagem para datetime com fuso horário UTC (aware datetime)
        data_msg = datetime.fromisoformat(data_msg.replace("Z", "+00:00"))  # Ajuste para o formato ISO 8601
        
        # Obter a data e hora atuais com fuso horário UTC (aware datetime)
        agora = datetime.now(timezone.utc)
        
        # Calcular a diferença de tempo
        diff = agora - data_msg
        return diff <= timedelta(hours=24)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False
    

def verificar_ultimos_7_dias(data_msg):
    try:
        # Converter a data da mensagem para datetime com fuso horário UTC (aware datetime)
        data_msg = datetime.fromisoformat(data_msg.replace("Z", "+00:00"))  # Ajuste para o formato ISO 8601
        
        # Obter a data e hora atuais com fuso horário UTC (aware datetime)
        agora = datetime.now(timezone.utc)
        
        # Calcular a diferença de tempo
        diff = agora - data_msg
        return diff <= timedelta(days=7)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False
    

def verificar_ultimo_mes(data_msg):
    try:
        # Converter a data da mensagem para datetime com fuso horário UTC (aware datetime)
        data_msg = datetime.fromisoformat(data_msg.replace("Z", "+00:00"))  # Ajuste para o formato ISO 8601
        
        # Obter a data e hora atuais com fuso horário UTC (aware datetime)
        agora = datetime.now(timezone.utc)
        
        # Calcular a diferença de tempo
        diff = agora - data_msg
        return diff <= timedelta(days=30)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False

### Gerando função dinamica

In [8]:
def create_json_file(nome_arquivo, funcao_offset):
    ids_processados = set()
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            if file.endswith('.json'):  # Verificar se é um arquivo JSON
                file_path = os.path.join(root, file)
                print(f"Lendo o arquivo: {file_path}")
                try:
                    # Ler o conteúdo do arquivo JSON
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        json_data = json.load(json_file)
                    
                    # Iterar sobre os itens da lista (cada item é uma mensagem)
                    for message_data in json_data:
                        # Extrair os dados necessários do JSON
                        message = message_data.get('message', '')
                        date = message_data.get('date', None)
                        media_path = message_data.get('media_path', None)
                        message_id = message_data.get('id', '')

                        # Verificar se o ID já foi processado ou se dados são inválidos
                        if not message or not date or not message_id:
                            continue
                        if message_id in ids_processados:
                            continue
                        if not funcao_offset(date):
                            continue

                        # Normalizar a mensagem
                        message_normalizada = normalizar_texto(message)
                        
                        # Identificar bairros mencionados na mensagem
                        bairros_mencionados = [
                            bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                            if bairro_normalizado in message_normalizada
                        ]
                        
                        # Se algum bairro foi identificado, salvar apenas o primeiro bairro
                        if bairros_mencionados:
                            primeiro_bairro = bairros_mencionados[0]
                            dados.append({
                                "id": message_id,
                                "mensagem": message,
                                "bairro": primeiro_bairro,
                                "data": date,
                                "media_path": media_path
                            })
                            ids_processados.add(message_id)  # Marcar o ID como processado
                except Exception as e:
                    print(f"Erro ao processar {file_path}: {e}")

    # Salvar os dados extraídos em um novo arquivo JSON
    output_folder = f"./{nome_arquivo}"  
    output_file = os.path.join(output_folder, f'{nome_arquivo}.json')
    os.makedirs(output_folder, exist_ok=True)

    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(dados, json_file, ensure_ascii=False, indent=4)

    print(f"Dados processados e salvos em {output_file}")

In [10]:
create_json_file('noticias_recentes', verificar_ultimas_24_horas)

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-15/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-15/metadata.json: 'str' object has no attribute 'get'
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-17/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-17/metadata.json: 'str' object has no attribute 'get'
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-18/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-18/metadata.json: 'str' object has no attribute 'get'
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-14/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-14/metadata.json: 'str' object has no attribute 'get'
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-19/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-19/metadata.json: 'str' object has no attribute 'get'
Lendo o arquivo: ../../downloa

## Agrupado por Bairro

In [7]:
def create_json_agg_bairro():
    agrupado_por_bairro = {}
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            if file.endswith('.json'):  # Verificar se é um arquivo JSON
                file_path = os.path.join(root, file)
                print(f"Lendo o arquivo: {file_path}")
                try:
                    # Ler o conteúdo do arquivo JSON
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        json_data = json.load(json_file)
                    
                    # Iterar sobre os itens do dicionário (cada chave é um ID de mensagem)
                    for message_id, message_data in json_data.items():
                        # Extrair os dados necessários
                        message = message_data.get('message', '')
                        date = message_data.get('date', None)
                        media_path = message_data.get('media_path', None)
                        
                        if message:
                            # Normalizar a mensagem
                            message_normalizada = normalizar_texto(message)
                            
                            # Identificar bairros mencionados na mensagem
                            bairros_mencionados = [
                                bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                                if bairro_normalizado in message_normalizada
                            ]
                            
                            # Se algum bairro foi identificado, salvar os dados agrupados
                            for bairro in bairros_mencionados:
                                if bairro not in agrupado_por_bairro:
                                    agrupado_por_bairro[bairro] = []  # Inicializa a lista para o bairro
                                
                                # Verifica se a mensagem já está no agrupamento para evitar duplicação
                                mensagem_existente = any(
                                    mensagem['id'] == message_id and mensagem['mensagem'] == message 
                                    for mensagem in agrupado_por_bairro[bairro]
                                )
                                
                                if not mensagem_existente:
                                    agrupado_por_bairro[bairro].append({
                                        "id": message_id,
                                        "mensagem": message,
                                        "data": date,
                                        "media_path": media_path
                                    })
                except Exception as e:
                    print(f"Erro ao processar {file_path}: {e}")

    # Criar um arquivo JSON separado para cada bairro
    output_folder = "../data/ceara/fortaleza/bairros"  # Pasta para salvar os arquivos
    os.makedirs(output_folder, exist_ok=True)

    for bairro, mensagens in agrupado_por_bairro.items():
        output_file = os.path.join(output_folder, f"{bairro}.json")
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(mensagens, json_file, ensure_ascii=False, indent=4)
        print(f"Arquivo criado para {bairro}: {output_file}")

In [8]:
create_json_agg_bairro()

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-15/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-17/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-18/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-14/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-19/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-13/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-16/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json: 'list' object has no attribute 'items'
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-27/metadata.json
Erro ao processar ../../downloads/Portalnoticiasceara/2024-12-27/metadata.json: 'list' object has no attribute 'items'
Arquivo criado para dende: ../data/ceara/fortaleza/bairros/d